<a href="https://colab.research.google.com/github/broadJeff/ColabsNotebooks/blob/main/DUDL_convolution_pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The notebook explains the concept of pooling in convolution

#### There are three main pooling techniques:
1. Max Pooling
2. Mean Pooling
3. Min Pooling

Pooling is a technique where we sequentially select an area of the input tensor and find the max/mean/min of all the values present in that area. After which, the window that selects the area moves to another area next to the selected area and performs the same operation (the specific number of steps that the windows moves is decide by the value of stride). Thus covering the entire tensor space. Every time the windows finds the intended value after the operation, it adds to a new tensor. Thus, leading to a creation of downsampled or low dimensional tensor